In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv('train.csv', header=0, sep=',', quotechar='"', on_bad_lines='skip') #on_bad_lines='skip'
print(df.head)
# df_test = pd.read_csv('test.csv', header=0, sep=',', quotechar='"', on_bad_lines='skip') #on_bad_lines='skip'
# print(df_test.head)

<bound method NDFrame.head of            text_id                                          full_text  \
0     0016926B079C  I think that students would benefit from learn...   
1     0022683E9EA5  When a problem is a change you have to let it ...   
2     00299B378633  Dear, Principal\n\nIf u change the school poli...   
3     003885A45F42  The best time in life is when you become yours...   
4     0049B1DF5CCC  Small act of kindness can impact in other peop...   
...            ...                                                ...   
3906  FFD29828A873  I believe using cellphones in class for educat...   
3907  FFD9A83B0849  Working alone, students do not have to argue w...   
3908  FFDC4011AC9C  "A problem is a chance for you to do your best...   
3909  FFE16D704B16  Many people disagree with Albert Schweitzer's ...   
3910  FFED00D6E0BD  Do you think that failure is the main thing fo...   

      cohesion  syntax  vocabulary  phraseology  grammar  conventions  
0          3.5     3.

In [3]:
df = df.drop(columns=['cohesion','vocabulary','phraseology','grammar','conventions'])
# df_test = df_test.drop(columns=['cohesion','vocabulary','phraseology','grammar','conventions'])

df['syntax'] = df['syntax'] * 2

print(list(df.columns))
print(df['syntax'].head)
print(df.shape)

['text_id', 'full_text', 'syntax']
<bound method NDFrame.head of 0       7.0
1       5.0
2       7.0
3       9.0
4       6.0
       ... 
3906    6.0
3907    8.0
3908    6.0
3909    9.0
3910    5.0
Name: syntax, Length: 3911, dtype: float64>
(3911, 3)


In [4]:
print(df['syntax'].describe())

count    3911.000000
mean        6.056507
std         1.288798
min         2.000000
25%         5.000000
50%         6.000000
75%         7.000000
max        10.000000
Name: syntax, dtype: float64


# Preprocessing

In [5]:
import contractions

df['full_text'] = df['full_text'].str.lower()
df['full_text'] = df['full_text'].str.lstrip()
df['full_text'] = df['full_text'].str.rstrip()
df['full_text'] = df['full_text'].str.replace(' +', ' ') #remove duplicate space - problem: might miss

df['full_text'] = df['full_text'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))

df['full_text'] = df['full_text'].str.replace(r'[^a-zA-Z\d\s:]', '') #removed special characters

C:\Users\lydia\AppData\Local\Temp\ipykernel_20384\415936517.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['full_text'] = df['full_text'].str.replace(' +', ' ') #remove duplicate space - problem: might miss
C:\Users\lydia\AppData\Local\Temp\ipykernel_20384\415936517.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['full_text'] = df['full_text'].str.replace(r'[^a-zA-Z\d\s:]', '') #removed special characters


In [6]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['full_text'] = df['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df.head(20))

         text_id                                          full_text  syntax
0   0016926B079C  think students would benefit learning homebeca...     7.0
1   0022683E9EA5  problem change let best matter happening chang...     5.0
2   00299B378633  dear principal change school policy grade b av...     7.0
3   003885A45F42  best time life become agree greatest accomplis...     9.0
4   0049B1DF5CCC  small act kindness impact people change people...     6.0
5   004AC288D833  dear principal school community center reasons...     8.0
6   005661280443  imagine could prove people good problem solver...     8.0
7   008DDDDD8E8D  think good idea estudnets commit career young ...     5.0
8   009BCCC61C2A  positive attitude key success agree anything l...     6.0
9   009F4E9310CB  asking one person advice help take strong choi...     6.0
10  00B21F9B726F  think good idea students commit career young a...     7.0
11  00BCADB373EF  positive attitude key success many people posi...     6.0
12  00D28152

# Gather TFIDF values

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit

train, test = train_test_split(df, test_size=0.2, random_state=2, shuffle=True, stratify=df['syntax'].values.astype(int))

train_labels = train['syntax'].values.astype(int).tolist()
test_labels = test['syntax'].values.astype(int).tolist()


In [8]:

vector = TfidfVectorizer(analyzer = 'word') #, ngram_range = (2,2)
tfidf_train = vector.fit_transform(train['full_text'])
tfidf_features = vector.get_feature_names_out()
tfidf_test = vector.transform(test['full_text'])

# Begin Running Linear Models on Data

In [9]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

perceptron_classifier = Perceptron(random_state=0) #max_iter=200

perceptron_classifier.fit(tfidf_train, train_labels)
y_pred = perceptron_classifier.predict(tfidf_test)

target_names = df['syntax'].unique() / 2
target_names = np.sort(target_names)
target_names_list = [str(i) for i in target_names]

# print(target_names_list, "target_names", len(target_names_list))
print(classification_report(test_labels, y_pred, target_names=target_names_list))
# print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         1.5       0.00      0.00      0.00         6
         2.0       0.22      0.13      0.17        82
         2.5       0.27      0.23      0.24       168
         3.0       0.33      0.36      0.35       250
         3.5       0.24      0.32      0.27       174
         4.0       0.18      0.13      0.15        78
         4.5       0.00      0.00      0.00        20
         5.0       0.00      0.00      0.00         3

    accuracy                           0.26       783
   macro avg       0.14      0.13      0.13       783
weighted avg       0.26      0.26      0.26       783



In [10]:
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm_classifier = make_pipeline(StandardScaler(with_mean=False), LinearSVC(random_state=0, tol=1e-5, max_iter=4000))
svm_classifier.fit(tfidf_train, train_labels)
y_pred = svm_classifier.predict(tfidf_test)
print(classification_report(test_labels, y_pred, target_names=target_names_list))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         1.5       0.00      0.00      0.00         6
         2.0       0.17      0.09      0.11        82
         2.5       0.27      0.26      0.26       168
         3.0       0.33      0.38      0.35       250
         3.5       0.24      0.32      0.27       174
         4.0       0.16      0.12      0.13        78
         4.5       0.00      0.00      0.00        20
         5.0       0.00      0.00      0.00         3

    accuracy                           0.27       783
   macro avg       0.13      0.13      0.13       783
weighted avg       0.25      0.27      0.25       783



c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [11]:
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(random_state=0).fit(tfidf_train, train_labels)

y_pred = logistic_regression_classifier.predict(tfidf_test)
print(classification_report(test_labels, y_pred, target_names=target_names_list))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         1.5       0.00      0.00      0.00         6
         2.0       0.50      0.04      0.07        82
         2.5       0.28      0.21      0.24       168
         3.0       0.34      0.65      0.44       250
         3.5       0.30      0.29      0.30       174
         4.0       0.00      0.00      0.00        78
         4.5       0.00      0.00      0.00        20
         5.0       0.00      0.00      0.00         3

    accuracy                           0.32       783
   macro avg       0.16      0.13      0.12       783
weighted avg       0.29      0.32      0.27       783



c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:133

# Try using word2vec

In [12]:
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim import utils
import gensim

model = api.load('word2vec-google-news-300') #load in the google model from the gensim library

In [13]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=2, shuffle=True)
train_labels = train['syntax'].values.astype(int).tolist()
test_labels = test['syntax'].values.astype(int).tolist()


In [14]:
train_sentences = []
word2vec_train_labels = []

# take each word in a review and average the word2vec vector 
for count, row in enumerate(train['full_text'].iteritems()):
    tokenized_words = word_tokenize(row[-1])
    words = []
    for w in tokenized_words:
        if w in model.key_to_index:
            word_vec = model.get_vector(w)
            words.append(word_vec)

    average_vals = np.average(words, axis=0)

    if average_vals.size == 300:
        # add averaged vector to train_sentences for training
        train_sentences.append(average_vals) 
        word2vec_train_labels.append(train_labels[count]) # get label -1 since model classifies 0-4 instead of 1-5


In [15]:
print("size of train sentences: ", len(train_sentences))
print("size of train labels: ", len(word2vec_train_labels))

size of train sentences:  3128
size of train labels:  3128


In [16]:
test_sentences = []
word2vec_test_labels = []

# take each word from test set reviews and average the word2vec vector
for count, row in enumerate(test['full_text'].iteritems()):
    tokenized_words = word_tokenize(row[-1])
    words = []
    for w in tokenized_words:
        if w in model.key_to_index:
            word_vec = model.get_vector(w)
            words.append(word_vec)
    
    average_vals = np.average(words, axis=0)
    if average_vals.size == 300:
        test_sentences.append(average_vals)
        word2vec_test_labels.append(test_labels[count])

In [17]:
print("size of train sentences: ", len(test_sentences))
print("size of train labels: ", len(word2vec_test_labels))
print(word2vec_test_labels[0:20])

size of train sentences:  783
size of train labels:  783
[8, 6, 2, 7, 5, 5, 6, 6, 7, 8, 5, 4, 6, 5, 8, 8, 6, 8, 4, 4]


In [18]:
# build the train and test lists for the fnn model
word2vec_train_labels_adjusted = [x - 2 for x in word2vec_train_labels]
word2vec_test_labels_adjusted = [x - 2 for x in word2vec_test_labels]
train_list = list(zip(train_sentences, word2vec_train_labels_adjusted))
test_list = list(zip(test_sentences, word2vec_test_labels_adjusted))

In [19]:
perceptron_classifier.fit(train_sentences, word2vec_train_labels)
y_pred = perceptron_classifier.predict(test_sentences)

print("Perceptron class accuracy results")
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6', 'class 7', 'class 8', 'class 9']
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))

Perceptron class accuracy results
              precision    recall  f1-score   support

         1.0  0.00000000 0.00000000 0.00000000         6
         1.5  0.00000000 0.00000000 0.00000000         7
         2.0  0.00000000 0.00000000 0.00000000        93
         2.5  0.27325581 0.56287425 0.36790607       167
         3.0  0.33410673 0.61016949 0.43178411       236
         3.5  0.00000000 0.00000000 0.00000000       171
         4.0  0.00000000 0.00000000 0.00000000        81
         4.5  0.25000000 0.05555556 0.09090909        18
         5.0  0.00000000 0.00000000 0.00000000         4

    accuracy                      0.30523627       783
   macro avg  0.09526250 0.13651103 0.09895547       783
weighted avg  0.16472913 0.30523627 0.21069952       783



c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [20]:
from sklearn.svm import SVC

svm_classifier = make_pipeline(StandardScaler(with_mean=False), SVC(kernel ='rbf', random_state=0, tol=1e-5, max_iter=4000))

svm_classifier.fit(train_sentences, word2vec_train_labels)
y_pred = svm_classifier.predict(test_sentences)

print("SVM class accuracy results")
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))


c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM class accuracy results
              precision    recall  f1-score   support

         1.0  0.00000000 0.00000000 0.00000000         6
         1.5  0.00000000 0.00000000 0.00000000         7
         2.0  0.00000000 0.00000000 0.00000000        93
         2.5  0.12500000 0.01796407 0.03141361       167
         3.0  0.30820106 0.98728814 0.46975806       236
         3.5  0.33333333 0.00584795 0.01149425       171
         4.0  0.00000000 0.00000000 0.00000000        81
         4.5  0.00000000 0.00000000 0.00000000        18
         5.0  0.00000000 0.00000000 0.00000000         4

    accuracy                      0.30268199       783
   macro avg  0.08517049 0.11234446 0.05696288       783
weighted avg  0.19235051 0.30268199 0.15079757       783



c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [21]:
logistic_regression_classifier = LogisticRegression(random_state=0).fit(train_sentences, word2vec_train_labels)

y_pred = logistic_regression_classifier.predict(test_sentences)
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))

              precision    recall  f1-score   support

         1.0  0.00000000 0.00000000 0.00000000         6
         1.5  0.00000000 0.00000000 0.00000000         7
         2.0  0.66666667 0.02150538 0.04166667        93
         2.5  0.26744186 0.13772455 0.18181818       167
         3.0  0.33167496 0.84745763 0.47675805       236
         3.5  0.28888889 0.15204678 0.19923372       171
         4.0  1.00000000 0.01234568 0.02439024        81
         4.5  0.00000000 0.00000000 0.00000000        18
         5.0  0.00000000 0.00000000 0.00000000         4

    accuracy                      0.32183908       783
   macro avg  0.28385249 0.13012000 0.10265187       783
weighted avg  0.40273063 0.32183908 0.23345863       783



c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:133

# Class balancing on linear models

In [22]:
#  class_weight="balanced"
logistic_regression_classifier = LogisticRegression(random_state=0, class_weight="balanced").fit(train_sentences, word2vec_train_labels)

y_pred = logistic_regression_classifier.predict(test_sentences)
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))

              precision    recall  f1-score   support

         1.0  0.00000000 0.00000000 0.00000000         6
         1.5  0.06578947 0.71428571 0.12048193         7
         2.0  0.23026316 0.37634409 0.28571429        93
         2.5  0.27722772 0.16766467 0.20895522       167
         3.0  0.28767123 0.08898305 0.13592233       236
         3.5  0.38095238 0.18713450 0.25098039       171
         4.0  0.20652174 0.23456790 0.21965318        81
         4.5  0.05504587 0.33333333 0.09448819        18
         5.0  0.00000000 0.00000000 0.00000000         4

    accuracy                      0.18646232       783
   macro avg  0.16705240 0.23359036 0.14624395       783
weighted avg  0.27959691 0.18646232 0.20025330       783



c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Deep learning on word2vec

In [23]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

c:\Users\lydia\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(300, 50),
            nn.ReLU(),
            nn.Linear(50, 16),
            nn.ReLU(),
            nn.Linear(16, 9)
        )

    def forward(self, x):
        return self.layers(x)
# initialize the NN
model_nn = Net()
print(model_nn)
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model_nn.parameters(), lr=0.005)

Net(
  (layers): Sequential(
    (0): Linear(in_features=300, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=9, bias=True)
  )
)


In [25]:
# number of subprocesses to use for data loading
from random import shuffle


num_workers = 0
# how many samples per batch to load
batch_size = 8

# obtain training indices that will be used for validation
num_train = len(train_sentences)
indices = list(range(num_train))
np.random.shuffle(indices)

valid_size = 0.2
# obtain training indices that will be used for validation
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders

train_loader = torch.utils.data.DataLoader(train_list, batch_size=batch_size,
     sampler=train_sampler, num_workers=num_workers)

valid_loader = torch.utils.data.DataLoader(train_list, batch_size=batch_size,
     sampler=valid_sampler, num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(test_list, batch_size=batch_size, 
    num_workers=num_workers)

In [26]:
for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0 #use validation to gage overfitting and best epoch length
    counter = 0 # use to keep track of number of occurrences

    # train the model 
    model_nn.train() # prep model for training
    for idx, (data,target) in enumerate(train_loader):
        counter += 1

        # clear the gradients of all optimized variables       
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_nn(data)
        # calculate the loss
        # print(target.shape)
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
        if counter%2000 == 0 or counter == len(train_loader):
            #if a print encounter, run the validation set to determine error
            for data, target in valid_loader:
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model_nn(data)
                # calculate the loss
                loss = criterion(output, target)
                # update running validation loss 
                valid_loss += loss.item()*data.size(0)
            
            valid_loss = valid_loss/len(valid_loader.dataset)
            print("Epoch {}...... Step: {}/{}....... Average Loss for Epoch: {},  Average Loss Overall: {}, Validation Loss: {}"
                  .format(epoch, counter, len(train_loader), train_loss/counter, train_loss/len(train_loader.dataset), valid_loss))
        



Epoch 0...... Step: 313/313....... Average Loss for Epoch: 17.06348803172858,  Average Loss Overall: 1.7074398190316642, Validation Loss: 0.40915515256659757
Epoch 1...... Step: 313/313....... Average Loss for Epoch: 15.808424586305222,  Average Loss Overall: 1.5818532274659638, Validation Loss: 0.38381511453167555
Epoch 2...... Step: 313/313....... Average Loss for Epoch: 14.931731071715918,  Average Loss Overall: 1.4941278214344893, Validation Loss: 0.366100372103474
Epoch 3...... Step: 313/313....... Average Loss for Epoch: 14.336988067093749,  Average Loss Overall: 1.434615493925941, Validation Loss: 0.35447391341714296
Epoch 4...... Step: 313/313....... Average Loss for Epoch: 13.952061482892631,  Average Loss Overall: 1.3960982238316475, Validation Loss: 0.3470439948236851
Epoch 5...... Step: 313/313....... Average Loss for Epoch: 13.705163766400883,  Average Loss Overall: 1.3713926658834643, Validation Loss: 0.34226490705824264
Epoch 6...... Step: 313/313....... Average Loss for

In [27]:

#build predict model for test set
def predict(model_nn, dataloader, y_true):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        # print(batch)
        data = torch.tensor(batch[0])   
        outputs = model_nn(data)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
        y_true.append(batch[1])
    return prediction_list


#build accuracy model for FNN
def accuracy(model_nn, predictions, y_true, batch, test_size):
    correct = 0
    # print(len(predictions))
    for i in range(0, len(predictions)):
        curBatch = predictions[i]
        # print(curBatch)
        curLabels = y_true[i]
        
        for j, pred in enumerate(curBatch):
            # print(pred, curLabels[j])           
            if pred == curLabels[j]:
                correct += 1
        # print(correct)
    
    print(correct)
    accuracy = correct / test_size
    return accuracy

In [28]:
test_size = len(test_sentences)

print("test size: ", test_size)
print("size of test sentences: ", len(test_sentences))

y_true = []
predictions = predict(model_nn,test_loader, y_true)

print(len(predictions))
print(predictions[0].shape)
print(len(y_true))

print(accuracy(model_nn, predictions, y_true, 20, test_size))

test size:  783
size of test sentences:  783
98
torch.Size([8])
98
236
0.30140485312899107


C:\Users\lydia\AppData\Local\Temp\ipykernel_20384\223528509.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(batch[0])


# Try XLNet BERT Model

In [29]:
!pip install transformers
!pip install sentencepiece


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
# https://medium.com/@yingbiao/text-classification-with-xlnet-in-action-869029246f7e
from transformers import XLNetForSequenceClassification
# model_file_address = './data/xlnet-base-cased'
model_file_address = "C:\\Users\\lydia\\Documents\\Class\\csci544\\english_language_learning_CS544\\data\\xlnet_base_cased"

# Will load config and weight with from_pretrained()
# Recommand download the model before using
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin"
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json" 
model = XLNetForSequenceClassification.from_pretrained(model_file_address,num_labels=2)


Some weights of the model checkpoint at C:\Users\lydia\Documents\Class\csci544\english_language_learning_CS544\data\xlnet_base_cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at C:\Users\lydia\Documents\Class\csci544\english_language_learning_CS544\data\xlnet_base_cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 

In [45]:
# from transformers import XLNetTokenizer, XLNetModel
#  http://restanalytics.com/2021-05-04-Fine-Tuning-XLNet-For-Sequence-Classification/
from transformers import XLNetTokenizer, XLNetLMHeadModel, XLNetModel, XLNetForMultipleChoice
import torch

xlnet_model_name = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(xlnet_model_name, do_lower_case=True)

# tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
model = XLNetForMultipleChoice.from_pretrained('xlnet-base-cased', num_labels=len(target_names_list))


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultipleChoice: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
token_lens = []
token_list = []

for txt in df['full_text'] :
  tokens = tokenizer.encode(txt, max_length=512, add_special_tokens=False)
  # max_length=512
  token_lens.append(len(tokens))
  token_list.append(tokens)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit

train, test = train_test_split(df, test_size=0.2, random_state=2, shuffle=True, stratify=df['syntax'].values.astype(int))

train_labels = train['syntax'].values.astype(int).tolist()
test_labels = test['syntax'].values.astype(int).tolist()


In [68]:
# input_ids = torch.tensor([tokenizer.encode(s , add_special_tokens=False) for s in token_list]).unsqueeze(0)

# outputs = model(token_list)

outputs = []
for i, token in enumerate(token_list):
    print(token, " is being processed")
    tokenized_encoding = torch.tensor(tokenizer.encode(token)).unsqueeze(0).flatten() #add_special_tokens=False
    print(tokenized_encoding, " is the tokenized encoding", tokenized_encoding.shape)
    outputs.append(model(tokenized_encoding, labels=target_names_list))
    print(outputs[i])
# last_hidden_states = outputs[0]
loss, classification_scores = outputs[:2]


[232, 466, 74, 1887, 1899, 192, 11730, 459, 133, 319, 907, 6256, 1162, 637, 2814, 3272, 480, 616, 1219, 3251, 192, 13646, 188, 297, 723, 1237, 4042, 216, 9509, 4232, 216, 12684, 338, 8933, 23, 232, 2779, 8322, 216, 338, 5868, 725, 115, 338, 197, 14095, 459, 560, 2814, 3134, 730, 1078, 192, 214, 3771, 3134, 466, 1044, 182, 17146, 297, 725, 182, 2284, 4042, 466, 5720, 195, 3535, 3706, 2400, 1864, 7273, 92, 280, 471, 297, 560, 2814, 214, 3706, 6192, 133, 931, 257, 216, 182, 6256, 133, 1237, 216, 192, 3251, 616, 1219, 1849, 2141, 25196, 176, 1486, 21700, 1075, 154, 281, 1326, 1132, 176, 714, 2804, 175, 4579, 162, 466, 1111, 3535, 466, 5199, 132, 5632, 1075]  is being processed
tensor([  232,   466,    74,  1887,  1899,   192, 11730,   459,   133,   319,
          907,  6256,  1162,   637,  2814,  3272,   480,   616,  1219,  3251,
          192, 13646,   188,   297,   723,  1237,  4042,   216,  9509,  4232,
          216, 12684,   338,  8933,    23,   232,  2779,  8322,   216,   338,
      

IndexError: tuple index out of range

In [ ]:

labels = torch.tensor(1).unsqueeze(0)  # Batch size 1

outputs = model(input_ids, labels=labels)
loss, classification_scores = outputs[:2]

In [38]:
print(last_hidden_states)

tensor([[[-16.3427, -35.1695, -34.6468,  ..., -31.4001, -22.6753, -21.7447],
         [-24.7520, -42.5470, -42.2781,  ..., -38.6128, -30.3361, -32.1718],
         [-27.2489, -43.5741, -43.4678,  ..., -40.3299, -37.2766, -36.0232],
         [-21.8894, -42.1254, -42.0676,  ..., -41.2019, -33.7251, -34.1798],
         [-21.6637, -39.6066, -39.5360,  ..., -36.6144, -32.3140, -30.9366],
         [-14.9458, -32.7998, -32.7879,  ..., -30.9330, -24.0895, -23.7962]]],
       grad_fn=<ViewBackward0>)
